In [85]:
# ------- SETTINGS -------
import os
import io
import numpy as np
import pandas as pd
from pathlib import PurePath, Path

# main variables
app_dir = '/some/path/to/app'
tree_path = '/Users/pierre/Downloads/muz_tree.txt'

# what is discarded
nope_dir = ['venv', 'static/js/lib', 'static/tify', 'static/node_modules', 'static/bootstrap']
nope_f = ['.DS_Store', 'bulma-tagsinput.min.css', '@mdi_materialdesignicons.min.css', '@fortawesome_all_tuned.css', 'bulma-pricingtable.min.css']
nope_ext = ['.zip', '.ico', '.svg', '.jpg', '.jpeg', '.png', '.ttf', '.woff', '.woff2', '.eot', '.OTF', '.pdf']



# ------- code -------
# Tree
# Cf. https://stackoverflow.com/questions/9727673/list-directory-tree-structure-in-python#answer-59109706
# prefix components:
space =  '    '
branch = '│   '
# pointers:
tee =    '├── '
last =   '└── '

def tree(dir_path: Path, prefix: str=''):
    """A recursive generator, given a directory Path object
    will yield a visual tree structure line by line
    with each line prefixed by the same characters
    """    
    contents = list(dir_path.iterdir())
    # contents each get pointers that are ├── with a final └── :
    pointers = [tee] * (len(contents) - 1) + [last]
    for pointer, path in zip(pointers, contents):
        if path.name != '.DS_Store':
            #if not 'static/node_modules' in path:
            yield prefix + pointer + path.name
            if path.is_dir(): # extend the prefix and recurse:
                extension = branch if pointer == tee else space 
                # i.e. space because last, └── , above so no more |
                yield from tree(path, prefix=prefix+extension)
            #else:
             #   continue

            
def types_and_paths(dir_path: Path, nope_dir: list, nope_f: list):
    '''Pulls the list of desired files and extensions to count lines from
    '''
    list_types = list()
    list_f = list()
    for dir_path ,dir_names, f_names in os.walk(app_dir):
        if len(f_names) > 0 and not any(s in dir_path for s in nope_dir):
            for el in f_names:
                if not any(s in el for s in nope_f):
                    list_f.append(Path(dir_path, el))
                    list_types.append(PurePath(el).suffix)
    return sorted(list(set(list_types))), sorted(list(set(list_f)))


# ---------------------------------------------------
print(f"--> Generate tree in file '{tree_path}' ...")
lines = list()
with open(tree_path, 'w', encoding='utf-8') as output:
    for l in tree(Path(app_dir)):
        lines.append(f'{l}\n')
    output.writelines(lines)
    output.close()
    print('Done')
# for l in lines:
    # print(l)

    
# ----------------------------
print('\n--> Count lines ...')
# crawl files
list_types, list_f = types_and_paths(app_dir, nope_dir, nope_f)

# prep totals
total = dict()
for t in list_types:
    if len(t) > 0:
        total[t] = [0,0]
    
# count 
for f in list_f:
    for k,v in total.items():
        if k == PurePath(f).suffix:
            total[k][0] += 1
            if not k in nope_ext:
                total[k][1] += len(io.open(Path(f)).readlines())
            pass
        else:
            continue

            
# ----------------------------
# print results
data = list()
for k,v in total.items():
    if k not in nope_ext:
        data.append([k, v[0], v[1]])
pd.DataFrame(data, columns=['Extension', 'Files', 'Lines']).sort_values('Lines', ascending=False)
# df = pd.DataFrame(data, columns=['Extension', 'Files', 'Lines'])
# print(df.to_string(index=False))

    

--> Generate tree in file '/Users/pierre/Downloads/muz_tree.txt' ...
Done

--> Count lines ...


,Extension,Files,Lines
8,.py,81,11778
3,.html,136,8452
1,.css,8,4406
7,.properties,1,758
4,.js,3,300
2,.csv,1,249
13,.xsl,3,224
9,.sh,2,209
5,.json,7,191
10,.txt,8,148
